<a href="https://colab.research.google.com/github/elizabethyoo/team_9/blob/data%2Fhousehold_pulse/Aggregate_HPS_state_level.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import glob
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import time
import re
from sklearn.preprocessing import OneHotEncoder
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [3]:
def w_avg(df, values, weights):
    d = df[values]
    w = df[weights]
    return (d * w).sum() / w.sum()*100

In [4]:
df = pd.read_csv('/content/drive/MyDrive/group_9/Datasets/household_pulse/cleaned_cmbn_household.csv', parse_dates = ['START_DATE','END_DATE' ])
df['MONTH'] = df['START_DATE'].dt.month

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
col_to_dummies = ['PWEIGHT','AGE_GRP', 'COVID_VACC_DOSE', 'EDUCATION', 'INCOME_BEFORE_TAX', 'RACE']

In [13]:
table = pd.get_dummies(df.set_index(['EST_ST', 'MONTH', 'YEAR'])[col_to_dummies], prefix=col_to_dummies.remove('PWEIGHT',)).reset_index()

In [14]:
table.head()

,EST_ST,MONTH,YEAR,PWEIGHT,AGE_GRP_18_24,AGE_GRP_25_39,AGE_GRP_40_49,AGE_GRP_50_59,AGE_GRP_60_69,AGE_GRP_70_up,AGE_GRP_less_18,COVID_VACC_DOSE_not_plan_all_dose,COVID_VACC_DOSE_plan_to_receive_all_doses,COVID_VACC_DOSE_received_all_doses,EDUCATION_bachelor_or_higher,EDUCATION_high_school_diploma_GED,EDUCATION_less_high school,EDUCATION_some_college_asscte_degree,"INCOME_BEFORE_TAX_$100,000_$149,999","INCOME_BEFORE_TAX_$150,000_$199,999","INCOME_BEFORE_TAX_$200,000_above","INCOME_BEFORE_TAX_$25,000_$34,999","INCOME_BEFORE_TAX_$35,000_$49,999","INCOME_BEFORE_TAX_$50,000_$74,999","INCOME_BEFORE_TAX_$75,000_$99,999","INCOME_BEFORE_TAX_Less_$25,000",RACE_Any_other_race_alone_or_mixed_race,"RACE_Asian,_alone",RACE_Black_alone,RACE_White_alone
0,1,9,2021,3311.293,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1,1,9,2021,1163.798,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
2,6,9,2021,2752.586,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1
3,1,9,2021,2818.850,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
4,1,9,2021,3311.293,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1


In [16]:
 col_to_agg= [ 'AGE_GRP_18_24', 'AGE_GRP_25_39','AGE_GRP_40_49', 'AGE_GRP_50_59',
              'AGE_GRP_60_69', 'AGE_GRP_70_up','AGE_GRP_less_18 ',
              'COVID_VACC_DOSE_not_plan_all_dose', 'COVID_VACC_DOSE_plan_to_receive_all_doses',
              'COVID_VACC_DOSE_received_all_doses', 'EDUCATION_bachelor_or_higher',
              'EDUCATION_high_school_diploma_GED', 'EDUCATION_less_high school',
              'EDUCATION_some_college_asscte_degree',
              'INCOME_BEFORE_TAX_$100,000_$149,999',
              'INCOME_BEFORE_TAX_$150,000_$199,999',
              'INCOME_BEFORE_TAX_$200,000_above', 'INCOME_BEFORE_TAX_$25,000_$34,999',
              'INCOME_BEFORE_TAX_$35,000_$49,999',
              'INCOME_BEFORE_TAX_$50,000_$74,999',
              'INCOME_BEFORE_TAX_$75,000_$99,999', 'INCOME_BEFORE_TAX_Less_$25,000',
              'RACE_Any_other_race_alone_or_mixed_race', 'RACE_Asian,_alone',
              'RACE_Black_alone', 'RACE_White_alone']

df_cat = pd.DataFrame()
for col in col_to_agg:
  df_wght = table.loc[table[col].notnull(),:]
  grp = df_wght.groupby(['EST_ST','YEAR','MONTH']).apply(w_avg, col, 'PWEIGHT').to_frame(col)
  df_cat = pd.concat([df_cat, grp], axis = 1, sort = False)

In [17]:
df_cat.head()

AGE_GRP_18_24  ...  RACE_White_alone
EST_ST YEAR MONTH                 ...                  
1      2020 4              5.684  ...            70.732
            5              7.033  ...            70.875
            6              9.057  ...            70.675
            7             10.691  ...            70.754
            8              9.548  ...            70.732

[5 rows x 26 columns]

In [18]:
df_cat.tail()

AGE_GRP_18_24  ...  RACE_White_alone
EST_ST YEAR MONTH                 ...                  
56     2021 5              8.608  ...            92.484
            6              6.945  ...            93.569
            7              7.511  ...            88.331
            8              8.164  ...            88.832
            9              6.540  ...            92.944

[5 rows x 26 columns]

In [5]:
 col_to_agg= ['FOOD_SCARCITY','HOUSING_INSECURITY', 'MALE','FEMALE',
              'HISPANIC_ORIGIN', 'RECVD_COVID_VACC', 
              'CHILDCARE_ISSUE']

df_grp = pd.DataFrame()
for col in col_to_agg:
  df_wght = df.loc[df[col].notnull(),:]
  grp = df_wght.groupby(['EST_ST','YEAR','MONTH']).apply(w_avg, col, 'PWEIGHT').to_frame(col)
  df_grp = pd.concat([df_grp, grp], axis = 1, sort = False)

In [6]:
df_grp.head()

FOOD_SCARCITY  ...  CHILDCARE_ISSUE
EST_ST YEAR MONTH                 ...                 
1      2020 4             10.411  ...              nan
            5             11.253  ...              nan
            6             13.731  ...              nan
            7             12.947  ...              nan
            8             14.083  ...              nan

[5 rows x 7 columns]

In [7]:
df_grp.tail()

FOOD_SCARCITY  ...  CHILDCARE_ISSUE
EST_ST YEAR MONTH                 ...                 
56     2021 5              5.306  ...            9.220
            6              9.028  ...            9.117
            7              8.489  ...           29.660
            8              9.261  ...           13.776
            9              8.054  ...           21.690

[5 rows x 7 columns]

In [21]:
df_combo = pd.concat([df_cat ,df_grp], axis = 1, sort = False)

In [22]:
df_combo.head()

AGE_GRP_18_24  ...  CHILDCARE_ISSUE
EST_ST YEAR MONTH                 ...                 
1      2020 4              5.684  ...              nan
            5              7.033  ...              nan
            6              9.057  ...              nan
            7             10.691  ...              nan
            8              9.548  ...              nan

[5 rows x 33 columns]

In [32]:
df_combo.reset_index(inplace=True)

In [47]:
state_number_ref= {
    '1':'Alabama',
    '2':'Alaska',
    '4':'Arizona',
    '5':'Arkansas',
    '6':'California',
    '8':'Colorado',
    '9':'Connecticut',
    '10':'Delaware',
    '11':'District of Columbia',
    '12':'Florida',
    '13':'Georgia',
    '15':'Hawaii',
    '16':'Idaho',
    '17':'Illinois',
    '18':'Indiana',
    '19':'Iowa',
    '20':'Kansas',
    '21':'Kentucky',
    '22':'Louisiana',
    '23':'Maine',
    '24':'Maryland',
    '25':'Massachusetts',
    '26':'Michigan',
    '27':'Minnesota',
    '28':'Mississippi',
    '29':'Missouri',
    '30':'Montana',
    '31':'Nebraska',
    '32':'Nevada',
    '33':'New Hampshire',
    '34':'New Jersey',
    '35':'New Mexico',
    '36':'New York',
    '37':'North Carolina',
    '38':'North Dakota',
    '39':'Ohio',
    '40':'Oklahoma',
    '41':'Oregon',
    '42':'Pennsylvania',
    '44':'Rhode Island',
    '45':'South Carolina',
    '46':'South Dakota',
    '47':'Tennessee',
    '48':'Texas',
    '49':'Utah',
    '50':'Vermont',
    '51':'Virginia',
    '53':'Washington',
    '54':'West Virginia',
    '55':'Wisconsin',
    '56':'Wyoming'
}

state_abbrev_dict= {
    "AL": "Alabama",
    "AK": "Alaska",
    "AZ": "Arizona",
    "AR": "Arkansas",
    "CA": "California",
    "CO": "Colorado",
    "CT": "Connecticut",
    "DE": "Delaware",
    "FL": "Florida",
    "GA": "Georgia",
    "HI": "Hawaii",
    "ID": "Idaho",
    "IL": "Illinois",
    "IN": "Indiana",
    "IA": "Iowa",
    "KS": "Kansas",
    "KY": "Kentucky",
    "LA": "Louisiana",
    "ME": "Maine",
    "MD": "Maryland",
    "MA": "Massachusetts",
    "MI": "Michigan",
    "MN": "Minnesota",
    "MS": "Mississippi",
    "MO": "Missouri",
    "MT": "Montana",
    "NE": "Nebraska",
    "NV": "Nevada",
    "NH": "New Hampshire",
    "NJ": "New Jersey",
    "NM": "New Mexico",
    "NY": "New York",
    "NC": "North Carolina",
    "ND": "North Dakota",
    "OH": "Ohio",
    "OK": "Oklahoma",
    "OR": "Oregon",
    "PA": "Pennsylvania",
    "RI": "Rhode Island",
    "SC": "South Carolina",
    "SD": "South Dakota",
    "TN": "Tennessee",
    "TX": "Texas",
    "UT": "Utah",
    "VT": "Vermont",
    "VA": "Virginia",
    "WA": "Washington",
    "WV": "West Virginia",
    "WI": "Wisconsin",
    "WY": "Wyoming",
}

abbre_dict = dict((value,key) for key,value in state_abbrev_dict.items())

In [50]:
df_combo['STATE'] = df_combo['EST_ST'].astype(str).map(state_number_ref)
df_combo['STATE_ABBR'] = df_combo['STATE'].astype(str).map(abbre_dict)

In [59]:
df_combo.head()

,EST_ST,YEAR,MONTH,AGE_GRP_18_24,AGE_GRP_25_39,AGE_GRP_40_49,AGE_GRP_50_59,AGE_GRP_60_69,AGE_GRP_70_up,AGE_GRP_less_18,COVID_VACC_DOSE_not_plan_all_dose,COVID_VACC_DOSE_plan_to_receive_all_doses,COVID_VACC_DOSE_received_all_doses,EDUCATION_bachelor_or_higher,EDUCATION_high_school_diploma_GED,EDUCATION_less_high school,EDUCATION_some_college_asscte_degree,"INCOME_BEFORE_TAX_$100,000_$149,999","INCOME_BEFORE_TAX_$150,000_$199,999","INCOME_BEFORE_TAX_$200,000_above","INCOME_BEFORE_TAX_$25,000_$34,999","INCOME_BEFORE_TAX_$35,000_$49,999","INCOME_BEFORE_TAX_$50,000_$74,999","INCOME_BEFORE_TAX_$75,000_$99,999","INCOME_BEFORE_TAX_Less_$25,000",RACE_Any_other_race_alone_or_mixed_race,"RACE_Asian,_alone",RACE_Black_alone,RACE_White_alone,FOOD_SCARCITY,HOUSING_INSECURITY,MALE,FEMALE,HISPANIC_ORIGIN,RECVD_COVID_VACC,CHILDCARE_ISSUE,STATE,STATE_ABBR
0,1,2020,4,5.684,31.468,17.525,16.585,14.933,12.348,1.457,0.000,0.000,0.000,23.643,42.442,2.212,31.703,10.759,4.754,1.968,11.872,10.321,18.878,8.790,15.813,3.719,1.746,23.803,70.732,10.411,4.330,47.242,52.758,3.999,nan,nan,Alabama,AL
1,1,2020,5,7.033,29.650,18.178,16.426,17.269,10.869,0.575,0.000,0.000,0.000,23.653,42.952,1.693,31.702,11.384,4.761,2.539,9.078,10.995,17.057,9.625,18.752,3.737,1.296,24.091,70.875,11.253,2.930,47.242,52.758,4.457,nan,nan,Alabama,AL
2,1,2020,6,9.057,27.398,16.748,18.827,17.043,10.412,0.516,0.000,0.000,0.000,23.645,41.907,2.735,31.713,10.084,4.177,3.148,11.942,11.515,16.274,8.840,20.502,3.908,1.350,24.067,70.675,13.731,2.401,47.242,52.758,5.964,nan,nan,Alabama,AL
3,1,2020,7,10.691,26.076,17.168,18.158,16.364,11.226,0.316,0.000,0.000,0.000,23.647,41.932,2.709,31.712,9.583,3.110,3.099,12.014,12.269,14.308,9.728,19.431,2.579,1.074,25.593,70.754,12.947,5.605,47.242,52.758,3.448,nan,nan,Alabama,AL
4,1,2020,8,9.548,26.712,16.009,18.143,17.261,11.252,1.075,0.000,0.000,0.000,23.666,41.907,2.734,31.694,8.161,3.113,2.594,6.971,8.431,13.646,9.634,15.709,2.205,0.922,26.141,70.732,14.083,2.814,47.242,52.758,3.433,nan,nan,Alabama,AL


In [60]:
df_combo.tail()

,EST_ST,YEAR,MONTH,AGE_GRP_18_24,AGE_GRP_25_39,AGE_GRP_40_49,AGE_GRP_50_59,AGE_GRP_60_69,AGE_GRP_70_up,AGE_GRP_less_18,COVID_VACC_DOSE_not_plan_all_dose,COVID_VACC_DOSE_plan_to_receive_all_doses,COVID_VACC_DOSE_received_all_doses,EDUCATION_bachelor_or_higher,EDUCATION_high_school_diploma_GED,EDUCATION_less_high school,EDUCATION_some_college_asscte_degree,"INCOME_BEFORE_TAX_$100,000_$149,999","INCOME_BEFORE_TAX_$150,000_$199,999","INCOME_BEFORE_TAX_$200,000_above","INCOME_BEFORE_TAX_$25,000_$34,999","INCOME_BEFORE_TAX_$35,000_$49,999","INCOME_BEFORE_TAX_$50,000_$74,999","INCOME_BEFORE_TAX_$75,000_$99,999","INCOME_BEFORE_TAX_Less_$25,000",RACE_Any_other_race_alone_or_mixed_race,"RACE_Asian,_alone",RACE_Black_alone,RACE_White_alone,FOOD_SCARCITY,HOUSING_INSECURITY,MALE,FEMALE,HISPANIC_ORIGIN,RECVD_COVID_VACC,CHILDCARE_ISSUE,STATE,STATE_ABBR
913,56,2021,5,8.608,22.979,17.001,16.579,18.735,13.374,2.724,0.000,0.000,0.000,27.103,29.508,1.136,42.254,12.592,4.687,3.442,6.437,8.301,15.841,11.516,8.969,4.662,2.027,0.827,92.484,5.306,1.097,50.580,49.420,6.510,64.353,9.220,Wyoming,WY
914,56,2021,6,6.945,22.778,22.160,14.229,22.056,10.784,1.048,0.000,0.000,0.000,27.106,35.741,0.844,36.309,14.006,3.521,3.438,7.896,7.785,15.541,10.372,10.121,5.004,0.410,1.017,93.569,9.028,1.978,50.580,49.420,8.028,62.831,9.117,Wyoming,WY
915,56,2021,7,7.511,26.491,18.581,17.458,14.633,14.699,0.628,0.570,1.163,64.560,27.103,24.028,0.933,47.936,18.183,5.576,3.167,9.006,10.179,13.320,7.783,14.035,9.737,1.323,0.610,88.331,8.489,1.059,50.580,49.420,7.300,66.970,29.660,Wyoming,WY
916,56,2021,8,8.164,27.052,17.255,16.316,19.941,11.130,0.142,0.401,3.628,61.144,27.104,30.377,0.190,42.330,14.421,4.561,3.024,6.418,8.778,14.739,12.105,13.950,8.577,1.293,1.297,88.832,9.261,1.434,50.580,49.420,6.884,67.012,13.776,Wyoming,WY
917,56,2021,9,6.540,27.550,17.503,15.963,20.661,10.160,1.623,0.000,4.408,65.803,27.103,25.580,0.799,46.517,12.758,4.498,4.527,4.622,10.221,11.492,11.305,14.420,4.942,0.700,1.415,92.944,8.054,1.420,50.580,49.420,7.489,72.666,21.690,Wyoming,WY


In [61]:
df_combo.to_csv('/content/drive/MyDrive/group_9/Datasets/household_pulse/household_state_agg.csv', index= False)